In [ ]:
# MP4 to PNG and PNG to MP4 Conversion Example
# ---------------------------------------------

# Import required libraries
import cv2
import os
import numpy as np
import glob
from IPython.display import display, HTML
from PIL import Image  # Import Image from PIL (Pillow) for image manipulation
import re
import shutil
from pathlib import Path
from datetime import datetime  # For timestamping output filenames

# Function to generate timestamped filenames
def get_timestamped_filename(base_name, extension):
    """
    Generate a filename with the current date and time.
    
    Parameters:
    -----------
    base_name : str
        Base filename without extension
    extension : str
        File extension (with or without dot)
        
    Returns:
    --------
    str
        Filename with timestamp appended
    """
    # Format: basename_YYYY_MM_DD_HHMMSS.extension
    timestamp = datetime.now().strftime("%Y_%m_%d_%H%M%S")
    
    # Ensure extension has a dot prefix
    if not extension.startswith('.'):
        extension = '.' + extension
        
    return f"{base_name}_{timestamp}{extension}"

# Define the conversion functions (from the previous cells)
# MP4 to PNG function
def convert_mp4_to_png(video_path, output_dir, prefix='frame', start_index=0, display_sample=True):
    """
    Convert an MP4 video to a sequence of PNG images.
    
    Parameters:
    -----------
    video_path : str
        Path to the input MP4 file
    output_dir : str
        Directory to save the PNG frames
    prefix : str, optional
        Prefix for the output filenames (default: 'frame')
    start_index : int, optional
        Starting index for frame numbering (default: 0)
    display_sample : bool, optional
        Whether to display a sample frame (default: True)
    
    Returns:
    --------
    int
        Number of frames extracted
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Check if video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return 0
    
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    print(f"Video FPS: {fps}")
    print(f"Total frames: {frame_count}")
    print(f"Resolution: {width}x{height}")
    
    # Read and save frames
    frame_idx = start_index
    sample_frame = None
    
    while True:
        # Read next frame
        ret, frame = cap.read()
        
        # Break if no more frames
        if not ret:
            break
            
        # Save the frame as PNG
        output_path = os.path.join(output_dir, f"{prefix}_{frame_idx:06d}.png")
        cv2.imwrite(output_path, frame)
        
        # Keep a sample frame for display (middle frame)
        if frame_idx == int(frame_count / 2) and display_sample:
            sample_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
        frame_idx += 1
    
    # Release the video capture object
    cap.release()
    
    # Display a sample frame if requested
    if display_sample and sample_frame is not None:
        print("Sample frame:")
        display(Image.fromarray(sample_frame))
    
    frames_extracted = frame_idx - start_index
    print(f"Extracted {frames_extracted} frames to {output_dir}")
    
    return frames_extracted, fps

# PNG to MP4 function
def convert_png_to_mp4(input_dir, output_path, pattern='*.png', fps=30, codec='mp4v', sort_numerically=True):
    """
    Convert a sequence of PNG images to an MP4 video.
    
    Parameters:
    -----------
    input_dir : str
        Directory containing the PNG image sequence
    output_path : str
        Path to save the output MP4 file
    pattern : str, optional
        Glob pattern to match the PNG files (default: '*.png')
    fps : int, optional
        Frames per second for the output video (default: 30)
    codec : str, optional
        FourCC codec to use (default: 'mp4v')
        Other options: 'avc1', 'H264', 'XVID'
    sort_numerically : bool, optional
        Whether to sort frames numerically by extracting numbers from filenames (default: True)
    
    Returns:
    --------
    bool
        True if conversion was successful, False otherwise
    """
    # Get list of PNG files
    png_files = glob.glob(os.path.join(input_dir, pattern))
    
    if not png_files:
        print(f"No PNG files found in {input_dir} matching pattern {pattern}")
        return False
    
    # Sort files
    if sort_numerically:
        # Extract numbers from filenames and sort based on that
        def extract_number(filename):
            numbers = re.findall(r'\d+', os.path.basename(filename))
            return int(numbers[-1]) if numbers else 0
        
        png_files.sort(key=extract_number)
    else:
        # Regular alphabetical sort
        png_files.sort()
    
    print(f"Found {len(png_files)} PNG files")
    
    # Read the first image to get dimensions
    first_img = cv2.imread(png_files[0])
    h, w, _ = first_img.shape
    
    print(f"Image dimensions: {w}x{h}")
    
    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*codec)
    out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))
    
    if not out.isOpened():
        print("Error: Could not create VideoWriter. Try a different codec.")
        return False
    
    # Add each image to the video
    for i, png_file in enumerate(png_files):
        if i % 100 == 0:  # Progress update every 100 frames
            print(f"Processing frame {i}/{len(png_files)}")
            
        img = cv2.imread(png_file)
        
        if img is None:
            print(f"Warning: Could not read image {png_file}")
            continue
            
        out.write(img)
    
    # Release the video writer
    out.release()
    
    print(f"Successfully created video: {output_path}")
    
    # Display the video in notebook if it exists
    if os.path.exists(output_path):
        video_path_html = os.path.abspath(output_path).replace('\\', '/')
        video_HTML = f"""
        <video width="640" height="480" controls>
            <source src="file://{video_path_html}" type="video/mp4">
            Your browser does not support the video tag.
        </video>
        """
        display(HTML(video_HTML))
        return True
    else:
        print("Error: Output video file was not created.")
        return False

# Usage Example 1: Convert MP4 to PNG sequence
# --------------------------------------------
# Set paths for input video and output frames
input_video = "input_video.mp4"  # Change this to your input video path
frames_dir = "extracted_frames"

# Uncomment to run the conversion
frames_count, original_fps = convert_mp4_to_png(input_video, frames_dir)

# Usage Example 2: Convert PNG sequence back to MP4
# ------------------------------------------------
# Generate timestamped output filename
output_video_base = "reconstructed_video"
output_video = get_timestamped_filename(output_video_base, "mp4")

# Set input frames directory
input_frames_dir = "./extracted_frames/upscayl_png_upscayl-standard-4x_4x"

# Run the conversion with the timestamped filename
# Use the original FPS from the video if available, or specify a value
# If original_fps isn't defined (because convert_mp4_to_png wasn't run),
# you'll need to specify the fps value manually, e.g., fps=30
#convert_png_to_mp4(input_frames_dir, output_video, pattern="frame_*.png", fps=24)  # Change fps if needed

In [ ]:
def reverse_convert_png_to_mp4(input_dir, output_path, pattern='*.png', fps=30, codec='mp4v', sort_numerically=True, reverse_order=False):
    """
    Convert a sequence of PNG images to an MP4 video.
    
    Parameters:
    -----------
    input_dir : str
        Directory containing the PNG image sequence
    output_path : str
        Path to save the output MP4 file
    pattern : str, optional
        Glob pattern to match the PNG files (default: '*.png')
    fps : int, optional
        Frames per second for the output video (default: 30)
    codec : str, optional
        FourCC codec to use (default: 'mp4v')
        Other options: 'avc1', 'H264', 'XVID'
    sort_numerically : bool, optional
        Whether to sort frames numerically by extracting numbers from filenames (default: True)
    reverse_order : bool, optional
        Whether to reverse the order of frames to play the video backwards (default: False)
    
    Returns:
    --------
    bool
        True if conversion was successful, False otherwise
    """
    # Get list of PNG files
    png_files = glob.glob(os.path.join(input_dir, pattern))
    
    if not png_files:
        print(f"No PNG files found in {input_dir} matching pattern {pattern}")
        return False
    
    # Sort files
    if sort_numerically:
        # Extract numbers from filenames and sort based on that
        def extract_number(filename):
            numbers = re.findall(r'\d+', os.path.basename(filename))
            return int(numbers[-1]) if numbers else 0
        
        png_files.sort(key=extract_number)
    else:
        # Regular alphabetical sort
        png_files.sort()
    
    # Reverse the order of frames if requested
    if reverse_order:
        png_files.reverse()
        print("Frame order reversed - video will play backwards")
    
    print(f"Found {len(png_files)} PNG files")
    
    # Read the first image to get dimensions
    first_img = cv2.imread(png_files[0])
    h, w, _ = first_img.shape
    
    print(f"Image dimensions: {w}x{h}")
    
    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*codec)
    out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))
    
    if not out.isOpened():
        print("Error: Could not create VideoWriter. Try a different codec.")
        return False
    
    # Add each image to the video
    for i, png_file in enumerate(png_files):
        if i % 100 == 0:  # Progress update every 100 frames
            print(f"Processing frame {i}/{len(png_files)}")
            
        img = cv2.imread(png_file)
        
        if img is None:
            print(f"Warning: Could not read image {png_file}")
            continue
            
        out.write(img)
    
    # Release the video writer
    out.release()
    
    print(f"Successfully created video: {output_path}")
    
    # Display the video in notebook if it exists
    if os.path.exists(output_path):
        video_path_html = os.path.abspath(output_path).replace('\\', '/')
        video_HTML = f"""
        <video width="640" height="480" controls>
            <source src="file://{video_path_html}" type="video/mp4">
            Your browser does not support the video tag.
        </video>
        """
        display(HTML(video_HTML))
        return True
    else:
        print("Error: Output video file was not created.")
        return False
# Generate timestamped output filename for the reversed video
output_video_reversed_base = "reversed_video"
output_video_reversed = get_timestamped_filename(output_video_reversed_base, "mp4")

# Run the conversion with the reversed frame order
reverse_convert_png_to_mp4(
    input_frames_dir, 
    output_video_reversed, 
    pattern="frame_*.png", 
    fps=24,
    reverse_order=True  # Set this to True to reverse the frame order
)

In [ ]:
# Image sequence to GIF function
def convert_images_to_gif(input_dir, output_path, pattern='*.png', fps=10, sort_numerically=True, 
                         optimize=True, loop=0, quality=95, display_output=True):
    """
    Convert a sequence of images to an animated GIF.
    
    Parameters:
    -----------
    input_dir : str
        Directory containing the image sequence
    output_path : str
        Path to save the output GIF file
    pattern : str, optional
        Glob pattern to match the image files (default: '*.png')
    fps : int, optional
        Frames per second for the output GIF (default: 10)
    sort_numerically : bool, optional
        Whether to sort frames numerically by extracting numbers from filenames (default: True)
    optimize : bool, optional
        Optimize the output file size (default: True)
    loop : int, optional
        Number of times the GIF should loop (0 means infinite, default: 0)
    quality : int, optional
        Image quality for the GIF (1-100, default: 95)
    display_output : bool, optional
        Whether to display the resulting GIF in the notebook (default: True)
    
    Returns:
    --------
    bool
        True if conversion was successful, False otherwise
    """
    from PIL import Image
    import glob
    import os
    import re
    from IPython.display import display, Image as IPyImage  # Fixed import name here
    
    # Get list of image files
    image_files = glob.glob(os.path.join(input_dir, pattern))
    
    if not image_files:
        print(f"No image files found in {input_dir} matching pattern {pattern}")
        return False
    
    # Sort files
    if sort_numerically:
        # Extract numbers from filenames and sort based on that
        def extract_number(filename):
            numbers = re.findall(r'\d+', os.path.basename(filename))
            return int(numbers[-1]) if numbers else 0
        
        image_files.sort(key=extract_number)
    else:
        # Regular alphabetical sort
        image_files.sort()
    
    print(f"Found {len(image_files)} image files")
    
    # Calculate frame duration in milliseconds
    duration = int(1000 / fps)
    
    # Load all images
    images = []
    first_img = None
    
    for i, img_path in enumerate(image_files):
        if i % 100 == 0:  # Progress update every 100 frames
            print(f"Processing frame {i}/{len(image_files)}")
            
        try:
            img = Image.open(img_path)
            
            # Store first image for reference
            if i == 0:
                first_img = img.copy()
            
            # Ensure all images have the same mode as the first one
            if img.mode != first_img.mode:
                img = img.convert(first_img.mode)
                
            images.append(img)
        except Exception as e:
            print(f"Warning: Could not read image {img_path}. Error: {e}")
    
    if not images:
        print("No valid images found to create GIF")
        return False
        
    print(f"Creating GIF with {len(images)} frames at {fps} FPS (duration: {duration}ms per frame)")
    
    try:
        # Save the GIF
        images[0].save(
            output_path,
            save_all=True,
            append_images=images[1:],
            optimize=optimize,
            duration=duration,
            loop=loop,
            quality=quality
        )
        print(f"Successfully created GIF: {output_path}")
        
        # Display the GIF in notebook if requested
        if display_output and os.path.exists(output_path):
            display(IPyImage(filename=output_path))  # Fixed variable name here
        
        return True
    except Exception as e:
        print(f"Error saving GIF: {e}")
        return False


# Usage Example: Convert PNG sequence to GIF
# ------------------------------------------
# Generate timestamped output filename
output_gif_base = "animated_sequence"
output_gif = get_timestamped_filename(output_gif_base, "gif")

# Set input frames directory - using the same directory from your example
input_frames_dir = "./extracted_frames/upscayl_png_upscayl-standard-4x_4x"

# Convert the PNG sequence to GIF
# Note: GIFs typically work better with lower FPS (10-15) than videos
convert_images_to_gif(input_frames_dir, output_gif, pattern="frame_*.png", fps=12)